In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pickle
import gc

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    BatchNormalization, GlobalAveragePooling2D, ReLU, BatchNormalization
)
from tensorflow.keras.applications import ResNet50, MobileNet, VGG16
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_curve
from sklearn.tree import DecisionTreeClassifier


2025-03-09 15:51:08.236239: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 15:51:08.246942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741510268.258684  605864 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741510268.262220  605864 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 15:51:08.276093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
input_shape = (32, 109, 1)
num_classes = 2

In [ ]:
X = np.load('../SavedFeatures/X_cqcc.npy')
y = np.load('../SavedFeatures/y_cqcc.npy')

pad_height = (3, 0)  
pad_width = (0, 0)   
X = np.pad(X, pad_width=((0, 0), pad_height, pad_width), mode='constant', constant_values=0)
X.shape

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
y_train_keras = to_categorical(y_train, num_classes=2)
y_test_keras = to_categorical(y_val, num_classes=2)
del X, y  
gc.collect()

22

In [ ]:
def model_vgg16(input_shape=(32, 109, 1), num_classes=2):
    base_model = VGG16(weights= None, include_top=False, input_shape=input_shape)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  
    x = BatchNormalization()(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)


def model_resnet(input_shape, num_classes):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output) 
    x = BatchNormalization()(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.4)(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.4)(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.2)(x)

    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)  
    x = Dropout(0.2)(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

def model_mobilenet(input_shape, num_classes,):
    base_model = MobileNet(input_shape=input_shape, weights=None, include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x) 
    x = BatchNormalization()(x)  
    x = Dropout(0.4)(x)  
    
    x = Dense(128)(x)
    x = ReLU(max_value=6)(x) 
    x = Dropout(0.3)(x)  
    
    x = Dense(64)(x)
    x = ReLU(max_value=6)(x) 
    x = Dropout(0.3)(x) 

    x = Dense(32)(x)
    x = ReLU(max_value=6)(x) 
    x = Dropout(0.2)(x)  
    
    output = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=output)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
cv_folds = 5
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
meta_train = np.zeros((X_train.shape[0], 3))
meta_test = np.zeros((X_test.shape[0], 3))

for train_idx, val_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train_keras[train_idx], y_train_keras[val_idx]
    
    models = {
        "vgg": model_vgg16(input_shape, num_classes),
        "resnet": model_resnet(input_shape, num_classes),
        "mobilenet": model_mobilenet(input_shape, num_classes)
    }   
    
    for name, model in models.items():
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_tr, y_tr, epochs=10, batch_size=16, verbose=0)
        meta_train[val_idx, list(models.keys()).index(name)] = np.argmax(model.predict(X_val), axis=1)
        del model  
        tf.keras.backend.clear_session()
        gc.collect()

final_models = {
    "vgg": model_vgg16(input_shape, num_classes),
    "resnet": model_resnet(input_shape, num_classes),
    "mobilenet": model_mobilenet(input_shape, num_classes)
}

for name, model in final_models.items():
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train_keras, epochs=10, batch_size=16, verbose=0)
    meta_test[:, list(final_models.keys()).index(name)] = np.argmax(model.predict(X_test), axis=1)
    del model
    tf.keras.backend.clear_session()
    gc.collect()

I0000 00:00:1741510278.432794  605864 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13473 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1741510281.701197  605953 service.cc:148] XLA service 0x71f0d80049a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741510281.701224  605953 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-03-09 15:51:21.755713: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741510282.086042  605953 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741510285.771576  605953 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step


2025-03-09 15:53:09.426295: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_17', 12 bytes spill stores, 36 bytes spill loads



64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


2025-03-09 15:57:15.962477: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_17', 12 bytes spill stores, 36 bytes spill loads



...

64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


2025-03-09 16:09:16.060342: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_17', 12 bytes spill stores, 36 bytes spill loads



64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step


2025-03-09 16:13:36.450466: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_17', 12 bytes spill stores, 36 bytes spill loads



100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [ ]:
meta_model = DecisionTreeClassifier(max_depth=20)
meta_model.fit(meta_train, y_train)
pickle.dump(meta_model, open("CQCC_ADD_DL_Stacking_model.pkl", "wb"))
del meta_train
gc.collect()

0

In [ ]:
meta_prob = meta_model.predict_proba(meta_test)[:, 1]
meta_predictions = (meta_prob > 0.5).astype(int)
accuracy = accuracy_score(y_test, meta_predictions)
f1 = f1_score(y_test, meta_predictions)
fpr, tpr, thresholds = roc_curve(y_test, meta_prob)
fnr = 1 - tpr
eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
eer = fpr[np.nanargmin(np.abs(fpr - fnr))]

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"EER: {eer:.4f}")



Accuracy: 0.9221
F1-Score: 0.8955
EER: 0.0798


0

In [ ]:
del meta_prob, meta_predictions
del accuracy, f1
del fpr, tpr, thresholds, fnr, eer_threshold, eer
del meta_test, meta_model, X_train, X_test, y_train, y_test  
gc.collect()
